In [ ]:
import numpy as np
import netCDF4 as nc
from wrf import getvar, destagger
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from bwave_ideal_wrf import epv_jet, qgpv_pert, wrf_fields, write_wrfinputd01


In [ ]:
nc_out = nc.Dataset('/p/work1/lloveras/adj_4km/rst_ctl/wrfrst_d01_2021-01-05_00_00_00')

title_str = "OUTPUT FROM IDEAL V3.6.1 PREPROCESSOR"
time_str = "2021-01-05_00:00:00"

ti = 0
dx = 4. # km
cent_length = 2500. # km

nx = 2000
ny = 1800
dx = 4.
hres_m = dx*1000.
rdx = 1/hres_m
rdy = 1/hres_m

dt = 20.
mp_physics = 17
ra_lw_physics = 0
ra_sw_physics = 0
sf_sfclay_physics = 1
sf_surface_physics = 0
bl_pbl_physics = 1
cu_physics = 0
num_soil_layers = 5
diff_opt = 1
km_opt = 4
damp_opt = 3
dampcoef = 0.4
khdif = 0.
kvdif = 0.
spec_bdy_width = 8
sf_lake_physics = 0
surface_input_source = 1
hypsometric_opt = 0
num_land_cat = 24
num_soil_cat = 16
ptop = 5000.
nz = 100

In [ ]:
u_out = getvar(nc_out,'U_2',timeidx=ti)
v_out = getvar(nc_out,'V_2',timeidx=ti)
t_out = getvar(nc_out,'T_2',timeidx=ti)
ph_out = getvar(nc_out,'PH_2',timeidx=ti)
phb_out = getvar(nc_out,'PHB',timeidx=ti)
p_out = getvar(nc_out,'P',timeidx=ti)
pb_out = getvar(nc_out,'PB',timeidx=ti)
mu_out = getvar(nc_out,'MU_2',timeidx=ti)
mub_out = getvar(nc_out,'MUB',timeidx=ti)
qv_out = getvar(nc_out,'QVAPOR',timeidx=ti)
tsk_out = getvar(nc_out,'TSK',timeidx=ti)
tmn_out = getvar(nc_out,'TMN',timeidx=ti)

t_temp = np.asarray(t_out[:,int(ny/2 - 1),int(nx/2 - 1)])
t_init_out = np.zeros((nz,ny,nx))
for k in range(nz):
    t_init_out[k,:,:] = t_temp[k]*np.ones((ny,nx))
    
t_init_new = t_out
t_init_new[:,:,:] = t_init_out
t_out = getvar(nc_out,'T_2',timeidx=ti)

psurf = p_out[0,:,:]
minp_ind = np.unravel_index(psurf.argmin(), psurf.shape)
cent_ind =  int(cent_length/dx)
rol_val = cent_ind - minp_ind[1]

u_roll = u_out.roll(west_east_stag=rol_val, roll_coords=False)
v_roll = v_out.roll(west_east=rol_val, roll_coords=False)
t_roll = t_out.roll(west_east=rol_val, roll_coords=False)
ph_roll = ph_out.roll(west_east=rol_val, roll_coords=False)
phb_roll = phb_out.roll(west_east=rol_val, roll_coords=False)
p_roll = p_out.roll(west_east=rol_val, roll_coords=False)
pb_roll = pb_out.roll(west_east=rol_val, roll_coords=False)
mu_roll = mu_out.roll(west_east=rol_val, roll_coords=False)
mub_roll = mub_out.roll(west_east=rol_val, roll_coords=False)
qv_roll = qv_out.roll(west_east=rol_val, roll_coords=False)
tsk_roll = tsk_out.roll(west_east=rol_val, roll_coords=False)
tmn_roll = tmn_out.roll(west_east=rol_val, roll_coords=False)
t_init_roll = t_init_new.roll(west_east=rol_val, roll_coords=False)

u_field = np.asarray(u_roll)
v_field = np.asarray(v_roll)
t = np.asarray(t_roll)
ph = np.asarray(ph_roll)
phb = np.asarray(phb_roll)
p = np.asarray(p_roll)
pb = np.asarray(pb_roll)
mu = np.asarray(mu_roll)
mub = np.asarray(mub_roll)
moist = np.asarray(qv_roll)
tsk = np.asarray(tsk_roll)
tmn = np.asarray(tmn_roll)
t_init = np.asarray(t_init_roll)

fnm = np.asarray(getvar(nc_out,'FNM'))
fnp = np.asarray(getvar(nc_out,'FNP'))
rdnw = np.asarray(getvar(nc_out,'RDNW'))
rdn = np.asarray(getvar(nc_out,'RDN'))
dnw = np.asarray(getvar(nc_out,'DNW'))
dn = np.asarray(getvar(nc_out,'DN'))
cfn = float(getvar(nc_out,'CFN'))
cfn1 = float(getvar(nc_out,'CFN1'))
cf1 = float(getvar(nc_out,'CF1'))
cf2 = float(getvar(nc_out,'CF2'))
cf3 = float(getvar(nc_out,'CF3'))
znw = np.asarray(getvar(nc_out,'ZNW'))
znu = np.asarray(getvar(nc_out,'ZNU'))

t_base = t[:,0,0]
u_base = u_field[:,0,0]
v_base = v_field[:,0,0]
qv_base = moist[:,0,0]


In [ ]:
nc_in = nc.Dataset('/p/work1/lloveras/adj_4km/in_files/wrfin_96h_test','w',format='NETCDF3_64BIT_OFFSET')
nc_in = write_wrfinputd01.write(nc_in, nx, ny, nz, hres_m, title_str, \
        time_str, u_field, v_field, t, ph, phb, t_init, mu, mub, p, pb, \
        fnm, fnp, rdnw, rdn, dnw, dn, t_base, cfn, cfn1, rdx, rdy, cf1, \
        cf2, cf3, tsk, u_base, v_base, qv_base, tmn, moist, znw, znu, \
        diff_opt, km_opt, damp_opt, dampcoef, khdif, kvdif, mp_physics, \
        ra_lw_physics, ra_sw_physics, sf_sfclay_physics, sf_surface_physics, \
        bl_pbl_physics, cu_physics, sf_lake_physics, surface_input_source, \
        hypsometric_opt, dt, num_land_cat, num_soil_layers, num_soil_cat, \
        spec_bdy_width, ptop)

nc_in.close()
